In [ ]:
!pip install mtcnn

In [ ]:
from mtcnn import MTCNN
import os
import numpy as np
import pandas as pd
import cv2 as cv2
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from tensorflow.keras.models import Model, load_model
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
pd.set_option('display.max_columns', 80)

### define a function that takes in an image and detects the faces in the image
### if there are multiple faces in the image it selects the faces having the most
### pixels in it and returns that as the cropped facial image

In [ ]:
def crop(img):
    # x, y, width, height = result['box']
    s=1.2
    height=img.shape[0]
    width=img.shape[1]    
    detector = MTCNN()
    data=detector.detect_faces(img) 
    if data==[]:
        return False, None
    else:
        for i, faces in enumerate(data): # iterate through all the faces found
            box=faces['box']  # get the box for each face
            biggest=0                    
            area = box[2] * box[3]
            if area>biggest:
                biggest=area
                bbox=box
            x,y,w,h=bbox  
            xn=int(x +w/2)-int(w * s/2)
            yn=int(y+h/2)- int(h * s/2)
            xen=int(x +w/2) + int(w * s/2)
            yen=int(y+h/2) + int(h * s/2)
            bbox[0]= 0 if bbox[0]<0 else bbox[0]
            xn=0 if xn<0 else xn
            yn=0 if yn<0 else yn
            xen= width if xen>width else xen
            yen= height if yen>height else yen
            img=img[yn:yen, xn:xen]            
            return True, img

### the classify function use the trained model to make predictions
sdir is he directory with the images to predict. It can contain a single image  
or multiple images of the SAME person. The function will process each image  
indepedently and return a prediction with the averaged probability  
csv_path is the path to the class_dict.csv file  
model_path is the path to the trained model   
crop_image is a boolean. If set to True the input images will automatically  
be cropped to contain just the face. If your images are already cropped  
set it to False

In [ ]:
def classify(sdir, csv_path,  model_path, crop_image = False):    
    # read in the csv file
    e=1.2
    class_df=pd.read_csv(csv_path)    
    img_height=int(class_df['height'].iloc[0])
    img_width =int(class_df['width'].iloc[0])
    img_size=(img_height, img_width)    
    scale=class_df['scale by'].iloc[0]     
    try: 
        s=int(scale)
        s2=s
        s1=0
    except:
        split=scale.split('-')
        s1=float(split[1])
        s2=float(split[0].split('*')[1]) 
    
    path_list=[]
    paths=os.listdir(sdir)
    for f in paths:
        path_list.append(os.path.join(sdir,f))
    print (' Model is being loaded- this will take about 10 seconds')
    model=load_model(model_path)
    image_count=len(path_list) 
    index_list=[] 
    prob_list=[]
    cropped_image_list=[]
    good_image_count=0
    for i in range (image_count):       
        img=plt.imread(path_list[i])        
        if crop_image == True:
            status, img=crop(img)            
        else:
            status=True
        if status== True:
            good_image_count +=1
            img=cv2.resize(img, img_size)             
            cropped_image_list.append(img)
            img=img*s2 - s1
            img=np.expand_dims(img, axis=0)
            p= np.squeeze (model.predict(img))           
            index=np.argmax(p)             
            prob=p[index]
            index_list.append(index)
            prob_list.append(prob)
    if good_image_count==1:
        print ( 'length of index list: ', len(index_list), 'index list [0]: ', index_list[0])
        class_name= class_df['class'].iloc[index_list[0]]
        probability= prob_list[0]
        img=cropped_image_list [0]/255 
        plt.title(class_name, color='blue', fontsize=16)
        plt.axis('off')
        plt.imshow(img)
        return class_name, probability
    elif good_image_count == 0:
        return None, None
    most=0
    for i in range (len(index_list)-1):
        key= index_list[i]
        keycount=0
        for j in range (i+1, len(index_list)):
            nkey= index_list[j]            
            if nkey == key:
                keycount +=1                
        if keycount> most:
            most=keycount
            isave=i             
    best_index=index_list[isave]    
    psum=0
    bestsum=0
    for i in range (len(index_list)):
        psum += prob_list[i]
        if index_list[i]==best_index:
            bestsum += prob_list[i]  
    img= cropped_image_list[isave]/255    
    class_name=class_df['class'].iloc[best_index]
    plt.title(class_name, color='blue', fontsize=16)
    plt.axis('off')
    plt.imshow(img)
    return class_name, bestsum/image_count

In [ ]:
predict_dir=r'../input/beauty-detection-data-set/images to predict'  
name='Elizabeth Hurley'
mloc=r'../input/beauty-detection-data-set/EfficientNetB1-beauty-100.0.h5'
csvloc=r'../input/beauty-detection-data-set/class_dict.csv'
result, probability=classify(predict_dir, csvloc,mloc,  crop_image=True)
print (f' {name} is predicted as being {result} with a probability of {probability * 100:5.2f} %')